In [33]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import sys
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random

# SEED SETTING
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)  # Ensures reproducibility

# Set root and paths
ROOT_PATH = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(ROOT_PATH)

from Training.Helper.dataPreprocessing import (
    add_time_features, add_lagged_features, add_rolling_features,
    sklearn_fit_transform, prepare_dataloader, rank_features_ccf,
    TRAIN_DATA_PATH_1990S
)
from Models.LSTM import LSTM

def create_direct_delta_sequences(X, y, seq_len=36, horizon=12):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_len - horizon):
        X_seq.append(X[i:i + seq_len])
        base = y[i + seq_len - 1]
        future = y[i + seq_len: i + seq_len + horizon]
        delta = future - base
        y_seq.append(delta)
    return np.array(X_seq), np.array(y_seq)

# === CONFIG ===
SEQ_LEN = 36
HORIZON = 12
BATCH_SIZE = 16
EPOCHS = 100
PATIENCE = 10
LR = 1e-3
TOP_K_FEATURES = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [34]:
# === LOAD & FEATURE ENGINEERING ===
df = pd.read_csv(TRAIN_DATA_PATH_1990S)
df["ds"] = pd.to_datetime(df["observation_date"], format="%m/%Y")
df = df.rename(columns={"fred_PCEPI": "y_original"})

df = add_time_features(df, "ds")
for k in [1, 2, 3, 4]:
    df[f"sin_{k}"] = np.sin(2 * np.pi * k * df["month"] / 12)
    df[f"cos_{k}"] = np.cos(2 * np.pi * k * df["month"] / 12)
df["pct_change"] = df["y_original"].pct_change()
df["momentum"] = df["pct_change"].diff()
df = add_lagged_features(df, ["y_original"], lags=[1, 6, 12])
df = add_rolling_features(df, "y_original", windows=[3, 6, 12])
df.dropna(inplace=True)

# === CCF SELECTION ===
df_numeric = df.select_dtypes(include=[np.number]).copy()
ccf_ranked = rank_features_ccf(df_numeric, targetCol="y_original")
selected_features = [col for col in list(ccf_ranked[:TOP_K_FEATURES]) if col in df.columns]
features = df[selected_features]
target_log = np.log1p(df["y_original"])

# === SCALE ===
features_scaled_list, x_scaler = sklearn_fit_transform(features, StandardScaler())
target_scaled_list, y_scaler = sklearn_fit_transform(target_log.to_frame(), StandardScaler())

X_scaled = features_scaled_list[0].values
y_scaled = target_scaled_list[0].values.flatten()

# === SEQUENCES ===
X_seq, y_seq = create_direct_delta_sequences(X_scaled, y_scaled, SEQ_LEN, HORIZON)
X_seq = X_seq.reshape(X_seq.shape[0], SEQ_LEN, -1)
y_seq = y_seq.reshape(y_seq.shape[0], HORIZON)

# === SPLIT ===
val_split = int(len(X_seq) * 0.8)
X_train, X_val = X_seq[:val_split], X_seq[val_split:]
y_train, y_val = y_seq[:val_split], y_seq[val_split:]

train_loader = prepare_dataloader(X_train, y_train, batch_size=BATCH_SIZE)
val_loader = prepare_dataloader(X_val, y_val, shuffle=False, batch_size=BATCH_SIZE)


2025-04-09 00:52:30,703 - INFO - Added time features: year, month, quarter. DataFrame shape: (408, 363)


In [ ]:
# === MODEL ===
model = LSTM(input_size=X_seq.shape[2], output_size=HORIZON).to(DEVICE)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Add Learning Rate Scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

# === TRAIN LOOP ===
print("\n Training LSTM with Delta Learning...")
best_loss = float("inf")
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(DEVICE)
        y_batch = y_batch.to(DEVICE)
        optimizer.zero_grad()
        pred = model(X_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    # === Validation ===
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(DEVICE)
            y_batch = y_batch.to(DEVICE)
            pred = model(X_batch)
            val_loss += criterion(pred, y_batch).item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1:02d}: Train MAE={avg_train_loss:.6f} | Val MAE={avg_val_loss:.6f}")

    # Step scheduler
    scheduler.step(avg_val_loss)

    # Optional: print current learning rate
    current_lr = optimizer.param_groups[0]["lr"]
    print(f"    → Learning rate: {current_lr:.6f}")

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_model_state = model.state_dict()
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping.")
            break

model.load_state_dict(best_model_state)

# === FINAL FORECAST (add deltas to base value) ===
model.eval()
with torch.no_grad():
    x_input = torch.tensor(X_seq[-1][np.newaxis], dtype=torch.float32).to(DEVICE)
    pred_delta = model(x_input).cpu().numpy().flatten()
    base_val = y_scaled[-1]
    pred_scaled = base_val + pred_delta

y_pred_rescaled = y_scaler.inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
y_pred_final = np.expm1(y_pred_rescaled)


/Users/natalieleung/miniforge3/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



 Training LSTM with Delta Learning...
Epoch 01: Train MAE=0.136012 | Val MAE=0.077422
    → Learning rate: 0.001000
Epoch 02: Train MAE=0.067680 | Val MAE=0.072237
    → Learning rate: 0.001000
Epoch 03: Train MAE=0.045388 | Val MAE=0.061155
    → Learning rate: 0.001000
Epoch 04: Train MAE=0.036974 | Val MAE=0.062949
    → Learning rate: 0.001000
Epoch 05: Train MAE=0.033609 | Val MAE=0.060846
    → Learning rate: 0.001000
Epoch 06: Train MAE=0.031757 | Val MAE=0.057116
    → Learning rate: 0.001000
Epoch 07: Train MAE=0.027929 | Val MAE=0.056622
    → Learning rate: 0.001000
Epoch 08: Train MAE=0.027212 | Val MAE=0.058799
    → Learning rate: 0.001000
Epoch 09: Train MAE=0.025158 | Val MAE=0.060713
    → Learning rate: 0.001000
Epoch 10: Train MAE=0.024334 | Val MAE=0.061584
    → Learning rate: 0.001000
Epoch 11: Train MAE=0.024647 | Val MAE=0.067601
    → Learning rate: 0.000500
Epoch 12: Train MAE=0.023073 | Val MAE=0.065682
    → Learning rate: 0.000500
Epoch 13: Train MAE=0.021

In [36]:
# === SAVE ===
save_path = os.path.join("..", "..", "Predictions", "LSTM.npy")
os.makedirs(os.path.dirname(save_path), exist_ok=True)
np.save(save_path, y_pred_final)
print(f"\nSaved improved LSTM with residuals to: {save_path}")


Saved improved LSTM with residuals to: ../../Predictions/LSTM.npy
